In [39]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score


In [40]:
test = pd.read_csv('../../Data/test.csv')
test_labels = pd.read_csv('../../Data/test_labels.csv')
test_filtered = pd.merge(test, test_labels)
test_filtered = test_filtered.drop(test_filtered.index[test_filtered['toxic'] == -1])

In [38]:
bert = pd.read_csv('../CatBoost/catboost_predictions.csv')
lreg = pd.read_csv('../LogisticRegression/predictions_csv_files/LR_precisions.csv')
nlb = pd.read_csv('../LogisticRegression/predictions_csv_files/NB-LR-BOW_precisions.csv')
nlt = pd.read_csv('../LogisticRegression/predictions_csv_files/NB-LR-TFIDF_precisions.csv')

labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
models = [bert, lreg]

The ensemble will perform better if the individual models have low correlations with each other.

In [41]:
for label in labels:
    print(label)
    print(np.corrcoef([x[label] for x in models]))

toxic
[[1.         0.71737539]
 [0.71737539 1.        ]]
severe_toxic
[[1.         0.46522711]
 [0.46522711 1.        ]]
obscene
[[1.         0.73659078]
 [0.73659078 1.        ]]
threat
[[1.        0.2553561]
 [0.2553561 1.       ]]
insult
[[1.         0.65851944]
 [0.65851944 1.        ]]
identity_hate
[[1.         0.37771564]
 [0.37771564 1.        ]]


In [42]:
# Create new predictions based on the other models
preds = models[0].copy()
preds.shape
for label in labels:
    sum = 0
    for x in models:
        sum += x[label]
    sum /= len(models)
    preds[label] = sum


# Calculate f1-score
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = preds[label] >= 0.5
    score = f1_score(test_filtered[label], pb)
    print(score)
    avg += score

avg /= len(labels)
print("Average f1-score:", avg)

toxic :
0.6607051235065602
severe_toxic :
0.39948453608247425
obscene :
0.6679368535655961
threat :
0.42045454545454547
insult :
0.6132075471698114
identity_hate :
0.41651031894934337
Average f1-score: 0.5297164874547218
